In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
import joblib
from scipy import sparse
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn import model_selection
from sklearn.model_selection import cross_val_score,cross_validate, KFold
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_curve, auc, classification_report,accuracy_score,precision_score,f1_score,recall_score,cohen_kappa_score,multilabel_confusion_matrix,make_scorer


In [2]:
# load data 
dataframe = pd.read_csv(r'C:\Users\dawns\Desktop\Master thesis\data\Features_level.csv')
#data=dataframe[['CONTENT_NEW','','SCORE']].copy()
df_data=dataframe[['CONTENT_NEW','WORDCUT','SCORE','LEVEL','Tokens','level1_tokens','level2_tokens','level3_tokens','level4_tokens','level5_tokens','level6_tokens','level7_tokens','char_num','high_level','primary_level','Connective_ratio','connective_SCORE','pre','connective_numbers']].copy()
df_data.head()

,CONTENT_NEW,WORDCUT,SCORE,LEVEL,Tokens,level1_tokens,level2_tokens,level3_tokens,level4_tokens,level5_tokens,level6_tokens,level7_tokens,char_num,high_level,primary_level,Connective_ratio,connective_SCORE,pre,connective_numbers
0,记得当年年纪小，家中兄弟姐妹数十多人。我排行倒最第二。父母亲为了养起这头家，日夜往外奔波劳累...,记得 当年 年纪 小 ， 家中 兄弟姐妹 数十 多人 。 我 排行 倒 最 第二 。 父母亲...,90,3,342,0,2,7,4,2,0,3,636,5,13,0.067251,2.984263,记得 当年 年纪 小 家中 兄弟姐妹 数十 多人 我 排行 倒 最 第二 父母亲 为了 养起...,23
1,《一封求职信》×××揭阳县西门外磐东镇肇沟乡广东省李文先生快乐家庭旅游公司负责人广东省李文先...,《 一封 求职信 》 × × × 揭阳县 西 门外 磐 东镇肇沟 乡 广东省 李文 先生 快...,80,3,194,0,0,0,3,0,0,5,403,5,3,0.051546,2.434391,一封 求职信 揭阳县 西 门外 磐 东镇肇沟 乡 广东省 李文 先生 快乐 家庭 旅游 公司...,10
2,我的外婆对我的影响非常大。从小，我父母便将我交给外婆带。他们是上班一族，平日无法照顾我。虽然...,我 的 外婆 对 我 的 影响 非常 大 。 从小 ， 我 父母 便 将 我 交给 外婆 带...,80,3,302,0,4,5,5,3,0,4,570,7,14,0.072848,3.791400,我 的 外婆 对 我 的 影响 非常 大 从小 我 父母 便 将 我 交给 外婆 带 他们 ...,22
3,“安乐死”究竟可不可取？在二十世纪的今天，“安乐死”已不是什么新话题。在不同的国家，已曾有人...,“ 安乐死 ” 究竟 可不 可取 ？ 在 二十世纪 的 今天 ， “ 安乐死 ” 已 不是 ...,95,3,273,1,0,4,3,1,1,8,588,10,8,0.084249,3.982972,安乐死 究竟 可不 可取 在 二十世纪 的 今天 安乐死 已 不是 什么 新 话题 在 不同...,23
4,关于安乐死这个问题，近来引起了广泛的讨论，有些人认为：一个人既既身罹不治之症，与其留在世上饱...,关于 安乐死 这个 问题 ， 近来 引起 了 广泛 的 讨论 ， 有些 人 认为 ： 一个 ...,85,3,186,1,1,5,4,1,0,4,364,5,11,0.096774,2.989940,关于 安乐死 这个 问题 近来 引起 了 广泛 的 讨论 有些 人 认为 一个 人 既 既身...,18


In [3]:
# Generating 'y' for classification as well as regression
y = df_data['SCORE']
#y

In [4]:
# generating x 
pre = df_data['pre']

In [5]:
#TFIDF Vectorizer
print(pre.shape)
vectorizer = TfidfVectorizer()
vectorizer.fit(pre)
vector = vectorizer.transform(pre)
print(vector.shape)


(3668,)
(3668, 36035)


In [6]:
#保存tf-idf特征 save tf-idf features
print(type(vector))

# save your model in disk
joblib.dump(vector, 'vector.pkl') 

# load your model
vector = joblib.load('vector.pkl') 

<class 'scipy.sparse._csr.csr_matrix'>


In [7]:
# 特征合并 feature concatenate 
feature_set1 = df_data[['connective_SCORE',]]
feature_set2 = vector
print(feature_set1.shape)
print(feature_set2.shape)
train_features = sparse.hstack((feature_set2, feature_set1))
print('=============')
print(train_features.shape)

(3668, 1)
(3668, 36035)
(3668, 36036)


In [8]:
# 划分训练集 ，测试集 train_test split
x_train, x_test, y_train, y_test = train_test_split(train_features, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)

(2934, 36036)
(2934,)


In [11]:
# evaluation 

def evaluation_scores(yt,yp):
    MSE = mean_squared_error(yt,yp)
    MAE = mean_absolute_error(yt,yp)
    R2 = r2_score(yt,yp)
    print('MSE:',MSE)
    print("================================================")
    print('MAE:',MAE)
    print("================================================")
    print('R2:',R2)
    print("================================================")
    
    

In [8]:
# linear regression 线性回归
from sklearn.linear_model import LinearRegression
linear_regressor = LinearRegression()
linear_regressor.fit(x_train, y_train)
y_pred = linear_regressor.predict(x_test)
r2 = r2_score(y_test,y_pred)
neg_mean_absolute_error = mean_absolute_error(y_test,y_pred)
neg_mean_squared_error = mean_squared_error(y_test,y_pred)
print('neg_mean_absolute_error:',neg_mean_absolute_error)
print('neg_mean_squared_error:',neg_mean_squared_error)
print('r2:',r2)



neg_mean_absolute_error: 7.395927744592785
neg_mean_squared_error: 89.72646613093579
r2: 0.3485797951795919


In [11]:
df= pd.DataFrame(y_pred)
print(df.min())
print(df.max())
df[(df >= 95)]=95
df[(df >= 90) & (df <95)]=90
df[(df >= 85) & (df < 90)]=85
df[(df >= 80) & (df < 85)]=80
df[(df >= 75) & (df < 80)]=75
df[(df >= 70) & (df < 75)]=70
df[(df >= 65) & (df < 70)]=65
df[(df >= 60) & (df < 65)]=60
df[(df >= 55) & (df < 60)]=55
df[(df >= 50) & (df < 55)]=50
df[(df >= 45) & (df < 50)]=45
df[ (df < 45)]=40
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)

0    40.0
dtype: float64
0    95.0
dtype: float64
qwkappa:
 0.5559152272281542


In [12]:
y_pred

array([65., 65., 60., 80., 80., 65., 65., 70., 65., 70., 70., 65., 65.,
       75., 65., 55., 65., 60., 70., 70., 65., 70., 70., 65., 80., 75.,
       60., 65., 80., 60., 55., 75., 75., 85., 65., 65., 70., 75., 60.,
       70., 75., 60., 55., 65., 60., 70., 70., 70., 60., 80., 60., 75.,
       90., 70., 65., 80., 55., 65., 80., 65., 75., 60., 70., 70., 65.,
       65., 65., 75., 90., 70., 65., 45., 75., 65., 65., 75., 70., 75.,
       75., 75., 75., 80., 65., 60., 65., 55., 75., 75., 60., 80., 65.,
       60., 75., 60., 85., 85., 60., 80., 45., 60., 70., 70., 80., 65.,
       70., 75., 90., 85., 85., 70., 85., 80., 65., 60., 65., 65., 80.,
       75., 70., 65., 75., 60., 65., 85., 65., 95., 75., 75., 55., 70.,
       90., 70., 65., 70., 80., 55., 65., 75., 70., 85., 65., 80., 60.,
       70., 55., 65., 55., 75., 70., 70., 70., 70., 80., 70., 85., 70.,
       70., 90., 80., 60., 65., 75., 75., 65., 75., 70., 85., 65., 75.,
       70., 75., 80., 40., 70., 85., 60., 70., 70., 65., 70., 75

In [13]:
# 随机森林 random forest

rf = ensemble.RandomForestRegressor()
params = {'n_estimators': [100], 'max_depth':[100], 'max_features':[10]}
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print('r2:',r2_score(y_test,y_pred))
print('mean_squared_error:',mean_squared_error(y_test,y_pred))
print('mean_absolute_error:',mean_absolute_error(y_test,y_pred))

r2: 0.39139739564769394
mean_squared_error: 83.82877989127076
mean_absolute_error: 7.160362332944077


In [14]:
df= pd.DataFrame(y_pred)
print(df.min())
print(df.max())
df[(df >= 95)]=95
df[(df >= 90) & (df <95)]=90
df[(df >= 85) & (df < 90)]=85
df[(df >= 80) & (df < 85)]=80
df[(df >= 75) & (df < 80)]=75
df[(df >= 70) & (df < 75)]=70
df[(df >= 65) & (df < 70)]=65
df[(df >= 60) & (df < 65)]=60
df[(df >= 55) & (df < 60)]=55
df[(df >= 50) & (df < 55)]=50
df[(df >= 45) & (df < 50)]=45
df[ (df < 45)]=40
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)

0    46.0
dtype: float64
0    89.45
dtype: float64
qwkappa:
 0.5108049743600471


In [9]:
#xgboost  xgboost处理回归
model = xgb.XGBRegressor()
params = {'n_estimators': [160], 'learning_rate': [0.1]}
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('R2:',r2_score(y_test,y_pred))
print('mean_squared_error:',mean_squared_error(y_test,y_pred))
print('mean_absolute_error:',mean_absolute_error(y_test,y_pred))

R2: 0.3977361405962314
mean_squared_error: 77.31495054691794
mean_absolute_error: 6.86582322081688


In [10]:
df= pd.DataFrame(y_pred)
print(df.min())
print(df.max())
df[(df >= 95)]=95
df[(df >= 90) & (df <95)]=90
df[(df >= 85) & (df < 90)]=85
df[(df >= 80) & (df < 85)]=80
df[(df >= 75) & (df < 80)]=75
df[(df >= 70) & (df < 75)]=70
df[(df >= 65) & (df < 70)]=65
df[(df >= 60) & (df < 65)]=60
df[(df >= 55) & (df < 60)]=55
df[(df >= 50) & (df < 55)]=50
df[(df >= 45) & (df < 50)]=45
df[ (df < 45)]=40
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)

0    39.797722
dtype: float32
0    93.848564
dtype: float32
qwkappa:
 0.5829622958198278


In [50]:
# 交叉验证 cross validation linear
cv_scores =[]
scores = cross_val_score(linear_regressor,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())


[array([0.26721891, 0.2437892 , 0.37174541, 0.29904049, 0.24336781,
       0.30014323, 0.27656403, 0.22895645, 0.29417619, 0.36340902])]
0.28884107301750395


In [51]:
# randomforest regressor
cv_scores =[]
scores = cross_val_score(rf,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())

[array([0.42580237, 0.41812314, 0.48111322, 0.36624081, 0.3784345 ,
       0.34818323, 0.40298443, 0.35252459, 0.41582634, 0.3759022 ])]
0.3965134837196277


In [52]:
#xgboost
cv_scores =[]
scores = cross_val_score(model,x_train,y_train,cv=10,scoring='r2')
cv_scores.append(scores)
print(cv_scores)
print(scores.mean())

[array([0.41705144, 0.35340054, 0.46571556, 0.41242091, 0.35365548,
       0.34951364, 0.32518356, 0.33000066, 0.37391405, 0.33973798])]
0.3720593825358806
